### **Implementation of DDPG (Deep Deterministic Policy Gradient) with HER (Hindsight Experience Replay)**

In [1]:
# To install the stable baseline
!pip install git+https://github.com/DLR-RM/stable-baselines3

  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-gqhunyat
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-gqhunyat
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit 5623d98f9d6bcfd2ab450e850c3f7b090aef5642
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu1

In [2]:
# To install highway-env
!pip install git+https://github.com/eleurent/highway-env

  Cloning https://github.com/eleurent/highway-env to /tmp/pip-req-build-1dvupazu
  Running command git clone --filter=blob:none --quiet https://github.com/eleurent/highway-env /tmp/pip-req-build-1dvupazu
  Resolved https://github.com/eleurent/highway-env to commit c7ee099195262e6eedfb63d88bd48a367818d59e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for highway-env: filename=highway_env-1.8.2-py3-none-any.whl size=107296 sha256=d017ee31a5ade6b65f62759bc120076287a26b85368abe4f24be30d6b50a3724
  Stored in directory: /tmp/pip-ephem-wheel-cache-mfsqwv0x/wheels/ef/2f/6d/3a938641a628175970386038dafd5a98053b743e9c0480c57d
Successfully built highway-env


## Import environment and stable baselines

In [3]:
import gymnasium as gym
import highway_env
import numpy as np

from stable_baselines3 import HerReplayBuffer, DDPG

In [4]:
env = gym.make("parking-v0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Training DDPG**

In [7]:
# Create the action noise object that will be used for exploration
her_kwargs = dict(n_sampled_goal=4, goal_selection_strategy='future')
model = DDPG('MultiInputPolicy',
             env,
             replay_buffer_class=HerReplayBuffer,
             replay_buffer_kwargs=her_kwargs,
             verbose=1,
             buffer_size=int(1e6),
             learning_rate=1e-3,
             gamma=0.95,
             batch_size=256,
             policy_kwargs=dict(net_arch=[256, 256, 256]))

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
import warnings
with warnings.catch_warnings():
  warnings.simplefilter("ignore")

  # Train for 2e5 steps
  model.learn(int(2e5))

Streaming output truncated to the last 5000 lines.
|    time_elapsed    | 2039     |
|    total_timesteps | 43793    |
| train/             |          |
|    actor_loss      | 1.18     |
|    critic_loss     | 0.00487  |
|    learning_rate   | 0.001    |
|    n_updates       | 44957    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.2     |
|    ep_rew_mean     | -9.44    |
|    success_rate    | 0.87     |
| time/              |          |
|    episodes        | 556      |
|    fps             | 21       |
|    time_elapsed    | 2043     |
|    total_timesteps | 43881    |
| train/             |          |
|    actor_loss      | 1.07     |
|    critic_loss     | 0.00422  |
|    learning_rate   | 0.001    |
|    n_updates       | 45045    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 31.7     |
|    ep_rew_mean     | -9.33   

KeyboardInterrupt: 

## Save the model


In [12]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive',force_remount=True)
path = r"/content/gdrive/MyDrive/Colab Notebooks/"

Mounted at /content/gdrive


In [13]:
# save the model
model.save(path + "ddpg_her_parking2e5")

# **Evaluate the agent**

In [15]:

# Visualization utils
import sys
from tqdm.notebook import trange
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg
!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
from utils import record_videos, show_videos

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,813 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main a

In [17]:
env = gym.make('parking-v0', render_mode='rgb_array')
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done = env.reset(), False
    for i in range(300):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        if done:
          break
env.close()
show_videos()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4




t:  79%|███████▉  | 49/62 [00:00<00:00, 130.07it/s, now=None]
                                                             

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
Moviepy - Building video /content/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-1.mp4




t:  97%|█████████▋| 85/88 [00:01<00:00, 57.02it/s, now=None]
                                                            

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-1.mp4
Moviepy - Building video /content/videos/rl-video-episode-2.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-2.mp4




t:  87%|████████▋ | 54/62 [00:00<00:00, 129.73it/s, now=None]
                                                             

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-2.mp4
